In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import Series
from pandas import read_csv
     

In [ ]:
!pip install rdflib 
import rdflib
!pip install SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

In [ ]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 
import pandas_profiling as pp

In [ ]:
!pip install pandas-profiling==3.2.0

In [ ]:
led_endpoint="http://data.open.ac.uk/sparql"

sparql_wd = SPARQLWrapper(led_endpoint)

# QUERIES  pre - war period 



* CQ1: What music was being played?
* CQ2: Who was listening to the music?
* CQ3: How did the instruments used change over time?
* CQ5: Where were the musicians coming from?
* CQ7: Where were the places (in which they played?)
* CQ10: How does the world of brass bands connect with that of elite music?
* CQ11: What did people ‘make of’, how did they react to, e.g. music?
* CQ12: Who was the author of a piece of music? e.g. can we identify it by its incipit?

* CQ14: What is the cause of a change, e.g. increasing number of concerts over time? # cannot be answered with a simple query, we have to identify the change
<br>

* CQ13: What was the country of origin of a piece of music? - we still have not retrieved info about this one. would it be interesting?

##CQ1: What music was being played?

In [ ]:
query_prewar= """PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
SELECT DISTINCT  ?lexp ?perf ?perflabel ?label ?songlabel ?genrelabel ?yoc
WHERE {
   ?lexp event:time [ rdfs:label ?when ; edtf:century edtfdata:Century.19 ; edtf:yearOfCentury/rdf:value ?yoc ].
   ?lexp  dc:subject ?perf.
   ?perf mo:performance_of ?song.
   ?perf rdfs:label ?perflabel.
   ?song rdfs:label ?songlabel.
   ?perf mo:genre ?genre.
   ?genre rdfs:label ?genrelabel.
   FILTER ( ?yoc >= 32 && ?yoc <= 38 ) 
}"""
# set the query
sparql_wd.setQuery(query_prewar)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

results

{'head': {'vars': ['lexp',
   'perf',
   'perflabel',
   'label',
   'songlabel',
   'genrelabel',
   'yoc']},
 'results': {'bindings': [{'lexp': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/lexp/1434481073543'},
    'perf': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/performance/http://data.open.ac.uk/led/music/Phantasy+in+F+sharp+minor/1434481071370/1434481071386'},
    'perflabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': "performance of 'Phantasy in F sharp minor'"},
    'songlabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Phantasy in F sharp minor'},
    'genrelabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Piano quartet'},
    'yoc': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '32'}},
   {'lexp': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/lexp/1448985100239'},
    'perf': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/performa

In [ ]:
#import pandas as pds

#df = pds.DataFrame(results['results']['bindings'])

#df.applymap(lambda x: x['value'])

In [ ]:
df = pd.DataFrame(columns=['lexp','perf','perflabel','songlabel','genrelabel', 'yoc'])
idx=0
for result in results["results"]["bindings"]:
    df.loc[idx] = [result["lexp"]["value"],result["perf"]["value"],result["perflabel"]["value"],result["songlabel"]["value"],result['genrelabel']['value'],result['yoc']['value']]
    idx+=1

In [ ]:
music = df.drop_duplicates()
music

,lexp,perf,perflabel,songlabel,genrelabel,yoc
0,http://data.open.ac.uk/led/lexp/1434481073543,http://data.open.ac.uk/led/performance/http://...,performance of 'Phantasy in F sharp minor',Phantasy in F sharp minor,Piano quartet,32
1,http://data.open.ac.uk/led/lexp/1448985100239,http://data.open.ac.uk/led/performance/6c8c036...,performance of 'Chamber music works by Schoenb...,Chamber music works by Schoenberg and his school,Chamber music,37
3,http://data.open.ac.uk/led/lexp/1426871249587,http://data.open.ac.uk/led/performance/http://...,performance of ''Our Hunting Fathers' op. 8','Our Hunting Fathers' op. 8,Classical music,36
4,http://data.open.ac.uk/led/lexp/1426871266847,http://data.open.ac.uk/led/performance/http://...,performance of ''Five Tudor Portraits'','Five Tudor Portraits',Classical music,36
5,http://data.open.ac.uk/led/lexp/1426871266847,http://data.open.ac.uk/led/performance/http://...,performance of ''Five Tudor Portraits'',Five Tudor Portraits,Classical music,36
...,...,...,...,...,...,...
418,http://data.open.ac.uk/led/lexp/1446380822350,http://data.open.ac.uk/led/performance/7715032...,performance of 'Macbeth',Macbeth,Opera,34
419,http://data.open.ac.uk/led/lexp/1446402813424,http://data.open.ac.uk/led/performance/45b7918...,performance of 'The Wreckers',The Wreckers,Opera,35
422,http://data.open.ac.uk/led/lexp/1446380822350,http://data.open.ac.uk/led/performance/7715032...,performance of 'Macbeth',Macbeth,Operatic,34
423,http://data.open.ac.uk/led/lexp/1446402813424,http://data.open.ac.uk/led/performance/45b7918...,performance of 'The Wreckers',The Wreckers,Operatic,35


In [ ]:
import pandas_profiling as pp

pp.ProfileReport(music)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

##Frequency of songs being played

In [ ]:
freq_song = """PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>

SELECT ?song ?songlabel ?musicfrequency WHERE {
SELECT DISTINCT ?song (COUNT(?song) as ?musicfrequency)
WHERE {
   ?lexp event:time [ rdfs:label ?when ; edtf:century edtfdata:Century.19 ; edtf:yearOfCentury/rdf:value ?yoc ].
   ?lexp  dc:subject ?perf.
   ?perf mo:performance_of ?song.
   ?perf rdfs:label ?perflabel.
   ?song rdfs:label ?songlabel.
   ?perf mo:genre ?genre.
   ?genre rdfs:label ?genrelabel.
   FILTER ( ?yoc >= 32 && ?yoc <= 38 ) 
}
GROUP BY ?song
}
ORDER BY DESC(?musicfrequency)"""

# set the query
sparql_wd.setQuery(freq_song)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

results

{'head': {'vars': ['song', 'songlabel', 'musicfrequency']},
 'results': {'bindings': [{'song': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/music/Der+Eing+des+Nibelungen/1439033179904'},
    'musicfrequency': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '24'}},
   {'song': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/music/Hansel+und+Gretel/1438108572879'},
    'musicfrequency': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '15'}},
   {'song': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/music/Beethoven+9/1447172558154'},
    'musicfrequency': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '14'}},
   {'song': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/music/Requiem/1388427609504'},
    'musicfrequency': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     

##CQ2: Who was listening to the music?

In [ ]:
listener = """PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>

SELECT DISTINCT ?listener ?givenName ?familyName ?birthdate ?nat ?occupation ?yoc

WHERE {
   ?lexp event:time [ rdfs:label ?when ; edtf:century edtfdata:Century.19 ; edtf:yearOfCentury/rdf:value ?yoc ].
   ?lexp dc:subject ?perf.
   ?perf mo:listener ?listener.
   ?listener foaf:givenName ?givenName ;
             foaf:familyName ?familyName ;
             schema:birthDate ?birthdate ;
             schema:nationality ?nat .
   ?listener db:occupation ?occupation .
   FILTER ( ?yoc >= 32 && ?yoc <= 38 ) 
}"""

# set the query
sparql_wd.setQuery(listener)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

results

{'head': {'vars': ['lexp',
   'listener',
   'givenName',
   'familyName',
   'birthdate',
   'nat',
   'occupation',
   'yoc']},
 'results': {'bindings': [{'lexp': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/lexp/1426341004158'},
    'listener': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/person/SavageRichardTemple1909-1996'},
    'givenName': {'type': 'literal', 'value': 'Richard Temple'},
    'familyName': {'type': 'literal', 'value': 'Savage'},
    'birthdate': {'type': 'uri',
     'value': 'http://data.open.ac.uk/time/edtf/1909-uu-uu'},
    'nat': {'type': 'uri', 'value': 'http://dbpedia.org/resource/England'},
    'occupation': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Writer'},
    'yoc': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '34'}},
   {'lexp': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/lexp/1426341004158'},
    'listener': {'type': 'uri',
     'value': 'http://bnb

###What to do with people that appear meny times in the results? How to remove this sort of duplicates? E.G. Igor Fjodorowitsch that has multiple nationalities (Igor' Fëdorovič Stravinskij - typos??) 

!Retrieve correct names from wikipedia - owl:SameAs???

In [ ]:
df = pd.DataFrame(columns=['lexp','listener','givenName','familyName','birthdate', 'nat', 'occupation', 'yoc'])
idx=0
for result in results["results"]["bindings"]:
    df.loc[idx] = [result["lexp"]["value"],result["listener"]["value"],result["givenName"]["value"],result["familyName"]["value"],result['birthdate']['value'],result['nat']['value'],result['occupation']['value'],result['yoc']['value']]
    idx+=1

In [ ]:
df

,lexp,listener,givenName,familyName,birthdate,nat,occupation,yoc
0,http://data.open.ac.uk/led/lexp/1426341004158,http://bnb.data.bl.uk/id/person/SavageRichardT...,Richard Temple,Savage,http://data.open.ac.uk/time/edtf/1909-uu-uu,http://dbpedia.org/resource/England,http://dbpedia.org/resource/Writer,34
1,http://data.open.ac.uk/led/lexp/1426341004158,http://bnb.data.bl.uk/id/person/SavageRichardT...,Richard Temple,Savage,http://data.open.ac.uk/time/edtf/1909-uu-uu,http://dbpedia.org/resource/England,http://data.open.ac.uk/led/clarinettist+music+...,34
2,http://data.open.ac.uk/led/lexp/1426341004158,http://bnb.data.bl.uk/id/person/SavageRichardT...,Richard Temple,Savage,http://data.open.ac.uk/time/edtf/1909-uu-uu,http://dbpedia.org/resource/England,http://data.open.ac.uk/led/music+librarian,34
3,http://data.open.ac.uk/led/lexp/1426341004158,http://bnb.data.bl.uk/id/person/SavageRichardT...,Richard Temple,Savage,http://data.open.ac.uk/time/edtf/1909-uu-uu,http://dbpedia.org/resource/England,http://dbpedia.org/resource/Clarinetist,34
4,http://data.open.ac.uk/led/lexp/1426871318822,http://bnb.data.bl.uk/id/person/SavageRichardT...,Richard Temple,Savage,http://data.open.ac.uk/time/edtf/1909-uu-uu,http://dbpedia.org/resource/England,http://dbpedia.org/resource/Writer,36
...,...,...,...,...,...,...,...,...
263,http://data.open.ac.uk/led/lexp/1446304716352,http://bnb.data.bl.uk/id/person/WoolfVirginia1...,Virginia,Woolf,http://reference.data.gov.uk/id/day/1882-01-25,http://dbpedia.org/resource/England,http://dbpedia.org/resource/Writer,32
264,http://data.open.ac.uk/led/lexp/1408110429181,http://bnb.data.bl.uk/id/person/PlowmanMax1883...,Max,Plowman,http://reference.data.gov.uk/id/day/1883-09-01,http://dbpedia.org/resource/England,http://dbpedia.org/resource/Soldier,37
265,http://data.open.ac.uk/led/lexp/1408110429181,http://bnb.data.bl.uk/id/person/PlowmanMax1883...,Max,Plowman,http://reference.data.gov.uk/id/day/1883-09-01,http://dbpedia.org/resource/England,http://dbpedia.org/resource/Writer,37
266,http://data.open.ac.uk/led/lexp/1408110429181,http://bnb.data.bl.uk/id/person/PlowmanMax1883...,Max,Plowman,http://reference.data.gov.uk/id/day/1883-09-01,http://dbpedia.org/resource/England,http://data.open.ac.uk/led/thing/pacifist,37


##CQ3: How did the instruments used change over time?

In [ ]:
instruments = """PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
SELECT DISTINCT   ?lexp ?perf ?perflabel ?instrument ?instrumentlabel ?genrelabel ?yoc
WHERE {
   ?lexp event:time [ rdfs:label ?when ; edtf:century edtfdata:Century.19 ; edtf:yearOfCentury/rdf:value ?yoc ].
   ?lexp  dc:subject ?perf.
   ?perf rdfs:label ?perflabel.
   ?perf mo:instrument ?instrument.
   ?instrument rdfs:label ?instrumentlabel.
   ?perf mo:genre ?genre.
   ?genre rdfs:label ?genrelabel.
   FILTER ( ?yoc >= 32 && ?yoc <= 38 ) 
}"""

# set the query
sparql_wd.setQuery(instruments)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

results

{'head': {'vars': ['lexp',
   'perf',
   'perflabel',
   'instrument',
   'instrumentlabel',
   'genrelabel',
   'yoc']},
 'results': {'bindings': [{'lexp': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/lexp/1414586405220'},
    'perf': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/performance/http://data.open.ac.uk/led/music/Suite+for+Chamber+Orchestra/1414586100698/1414586100712'},
    'perflabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': "performance of 'Suite for Chamber Orchestra'"},
    'instrument': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Orchestra'},
    'instrumentlabel': {'type': 'literal', 'value': 'Orchestra'},
    'genrelabel': {'type': 'literal', 'value': 'Chamber music'},
    'yoc': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '35'}},
   {'lexp': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/lexp/1414586405220'},
    'perf': {'type': 'uri',
     'value

In [ ]:
import pandas as pds
df = pds.DataFrame(results['results']['bindings'])
df.applymap(lambda x: x['value'])

,lexp,perf,perflabel,instrument,instrumentlabel,genrelabel,yoc
0,http://data.open.ac.uk/led/lexp/1414586405220,http://data.open.ac.uk/led/performance/http://...,performance of 'Suite for Chamber Orchestra',http://dbpedia.org/resource/Orchestra,Orchestra,Chamber music,35
1,http://data.open.ac.uk/led/lexp/1414586405220,http://data.open.ac.uk/led/performance/http://...,performance of 'Suite for Chamber Orchestra',http://dbpedia.org/resource/Orchestra,Orchestra,Chamber music,35
2,http://data.open.ac.uk/led/lexp/1414586405220,http://data.open.ac.uk/led/performance/http://...,performance of 'Suite for Chamber Orchestra',http://dbpedia.org/resource/Orchestra,Orchestra,Chamber music,35
3,http://data.open.ac.uk/led/lexp/1414586405220,http://data.open.ac.uk/led/performance/http://...,performance of 'Suite for Chamber Orchestra',http://dbpedia.org/resource/Orchestra,Orchestra,Chamber music,35
4,http://data.open.ac.uk/led/lexp/1431351729781,http://data.open.ac.uk/led/performance/http://...,performance of 'Shostakovich's Fourth Symphony',http://dbpedia.org/resource/Orchestra,Orchestra,Classical music,36
...,...,...,...,...,...,...,...
966,http://data.open.ac.uk/led/lexp/1450115937591,http://data.open.ac.uk/led/performance/4465299...,performance of 'Mikrokosmos',http://dbpedia.org/resource/Piano,Piano,Classical music,37
967,http://data.open.ac.uk/led/lexp/1450115937591,http://data.open.ac.uk/led/performance/a1d8eff...,performance of 'Children's choruses',http://dbpedia.org/resource/Human_voice,Human voice,Classical music,37
968,http://data.open.ac.uk/led/lexp/1450115937591,http://data.open.ac.uk/led/performance/a1d8eff...,performance of 'Children's choruses',http://dbpedia.org/resource/Human_voice,Human voice,Classical music,37
969,http://data.open.ac.uk/led/lexp/1450115937591,http://data.open.ac.uk/led/performance/4465299...,performance of 'Mikrokosmos',http://dbpedia.org/resource/Piano,Piano,Classical music,37


##CQ5: Where were the musicians coming from?

In [ ]:
musicians = """PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
  
SELECT DISTINCT  ?perf ?artist ?artistlabel ?natlabel ?yoc
WHERE { 
  ?lexp event:time [ rdfs:label ?when ; edtf:century edtfdata:Century.19 ; edtf:yearOfCentury/rdf:value ?yoc ].
  ?lexp dc:subject ?perf.
  ?perf mo:performer ?artist.
  ?artist rdfs:label ?artistlabel.
  ?artist schema:nationality ?nat.
  ?nat rdfs:label ?natlabel.
  FILTER ( ?yoc >= 32 && ?yoc <= 38 )
}"""

# set the query
sparql_wd.setQuery(musicians)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

results

##CQ7: Where were the places (in which they played?)

In [ ]:
place = """PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>  
SELECT DISTINCT  ?perf ?place ?placelabel ?lat ?long ?yoc
WHERE { 
  ?lexp event:time [ rdfs:label ?when ; edtf:century edtfdata:Century.19 ; edtf:yearOfCentury/rdf:value ?yoc ].
  ?lexp dc:subject ?perf.
  ?perf event:place ?place.
  ?place rdfs:label ?placelabel.
  SERVICE <https://dbpedia.org/sparql/>
     { ?place geo:lat ?lat .
       ?place geo:long ?long .}
  FILTER ( ?yoc >= 32 && ?yoc <= 38 )
}"""

# set the query
sparql_wd.setQuery(place)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

results

{'head': {'vars': ['perf', 'place', 'placelabel', 'lat', 'long', 'yoc']},
 'results': {'bindings': [{'perf': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/performance/7045ccc414dedaadb1d874cfe548a78e/1446306081238'},
    'place': {'type': 'uri', 'value': 'http://dbpedia.org/resource/London'},
    'placelabel': {'type': 'literal', 'value': 'London'},
    'lat': {'datatype': 'http://www.w3.org/2001/XMLSchema#float',
     'type': 'literal',
     'value': '51.5072'},
    'long': {'datatype': 'http://www.w3.org/2001/XMLSchema#float',
     'type': 'literal',
     'value': '-0.1275'},
    'yoc': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '33'}},
   {'perf': {'type': 'uri',
     'value': 'http://data.open.ac.uk/led/performance/6c7ec733c1945c87eda522364004be5f/1449058285194'},
    'place': {'type': 'uri', 'value': 'http://dbpedia.org/resource/London'},
    'placelabel': {'type': 'literal', 'value': 'London'},
    'lat': {'datatype'

# Sentiment analysis with [Scikit-learn](https://scikit-learn.org/stable/):

❗ Made an attempt but all is classified as positive. maybe we should use more sentiment categories. <br>
Using a training dataset downloaded from [kaggle](https://www.kaggle.com/nltkdata/movie-review):

In [ ]:
import pandas as pd
df = pd.read_csv('movie_review.csv')

In [ ]:
df.fillna("")

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos
...,...,...,...,...,...,...
28673,8,cv871,24888,8,"reluctantly , he sets up nyah to be rescued fr...",pos
28674,8,cv871,24888,9,ethan must race against time to keep ambrose f...,pos
28675,8,cv871,24888,10,although the first mission : impossible film w...,pos
28676,8,cv871,24888,11,that problem has been rectified in the sequel ...,pos


In [ ]:
X = df['text'] 
y = df['tag']

In [ ]:
from numpy import random,array

In [ ]:
random.seed(0)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer(ngram_range=(1,2)) 
X=vect.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)


In [ ]:
from sklearn.naive_bayes import BernoulliNB

In [ ]:
model = BernoulliNB()

In [ ]:
model.fit(X_train, y_train)

BernoulliNB()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
#errori = accuracy_score(y_test, p_test)
#print(errori)

How to analyze a single query:

In [ ]:
q=array(['good wonderful film']) 
q=vect.transform(q)

In [ ]:
print(model.predict(q))

['pos']


## CQ11 What did people ‘make of’, how did they react to, e.g. music?

In [ ]:
# CQ 11
#retrieve texts for doing sentiment analysis
query = """
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
  
SELECT DISTINCT  ?songlabel ?text ?date ?description ?when ?yoc
WHERE { 
  ?lexp event:time [ rdfs:label ?when ; edtf:century edtfdata:Century.19 ; edtf:yearOfCentury/rdf:value ?yoc ].
  
  ?lexp  dc:subject ?perf.
  ?perf mo:performance_of ?song.
  ?song rdfs:label ?songlabel.
  ?lexp led:has_evidence_text ?text.
  FILTER ( ?yoc >= 32 && ?yoc <= 38 ) 
}
"""
# set the query
sparql_wd.setQuery(query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

In [ ]:
# easier way to obtain a dataframe
import pandas as pds
df_text = pds.DataFrame(results['results']['bindings'])
df_text.applymap(lambda x: x['value']) #drop the duplicates! maybe selecting the perf uri instead of songlabel

,songlabel,text,when,yoc
0,Die Meistersinger,Listen to a Wagner Prom after dinner on a very...,"19 September, 1932",32
1,G��tterd��mmerung,Listen to a Wagner Prom after dinner on a very...,"19 September, 1932",32
2,Der ing des Nibelungen,In the April season Flagstad was back to delig...,"April, 1937",37
3,Der Ring des Nibelungen,In the April season Flagstad was back to delig...,"April, 1937",37
4,Ring of the Nibelungen,In the April season Flagstad was back to delig...,"April, 1937",37
...,...,...,...,...
379,String Quartet No.9,In contrast with most musical audiences this s...,"5 October, 1937",37
380,unspecified songs,R.C.M. Jubilee. concert at 3.0 attended by T.M...,"9 May, 1933, 03:00 PM",33
381,Wasps,R.C.M. Jubilee. concert at 3.0 attended by T.M...,"9 May, 1933, 03:00 PM",33
382,"The Planets, Op. 32 - Jupiter",R.C.M. Jubilee. concert at 3.0 attended by T.M...,"9 May, 1933, 03:00 PM",33


In [ ]:
# sentiment analysis with scikit learn -> they're all positive -> shall we do another kind of analysis?
for result in results["results"]["bindings"]:
    q =array([result["text"]["value"]])
    q=vect.transform(q)
    print(result["songlabel"]["value"], "\n",model.predict(q), " ->",[result["text"]["value"]])

In [ ]:
df_sent = pd.DataFrame(columns=['songlabel','sentiment'])
idx=0
for result in results["results"]["bindings"]:
    
    q =array([result["text"]["value"]])
    q=vect.transform(q)
    sent=str(model.predict(q)).replace("[","").replace("]","")
    df_sent.loc[idx] = [result["songlabel"]["value"],sent]
    idx+=1
    

In [ ]:
df_sent

,songlabel,sentiment
0,Die Meistersinger,'pos'
1,G��tterd��mmerung,'pos'
2,Der ing des Nibelungen,'pos'
3,Der Ring des Nibelungen,'pos'
4,Ring of the Nibelungen,'pos'
...,...,...
379,String Quartet No.9,'pos'
380,unspecified songs,'pos'
381,Wasps,'pos'
382,"The Planets, Op. 32 - Jupiter",'pos'


## CQ12: Who was the author of a piece of music? e.g. can we identify it by its incipit?

In [ ]:
#CQ12: Who was the author of a piece of music?
# retrieve the composers of performances
query = """
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
  
SELECT DISTINCT  ?perf ?songlabel ?author ?yoc
WHERE { 
  ?lexp event:time [ rdfs:label ?when ; edtf:century edtfdata:Century.19 ; edtf:yearOfCentury/rdf:value ?yoc ]. 
  ?lexp  dc:subject ?perf.
  ?perf mo:performance_of ?song.
  ?song rdfs:label ?songlabel.
  ?song mo:composer ?author.
  FILTER ( ?yoc >= 32 && ?yoc <= 38 )
  }
"""
# set the query
sparql_wd.setQuery(query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results_author = sparql_wd.query().convert()


In [ ]:
df_author = pds.DataFrame(results_author['results']['bindings'])
df_author.applymap(lambda x: x['value'])

,perf,songlabel,author,yoc
0,http://data.open.ac.uk/led/performance/http://...,Die Meistersinger,http://dbpedia.org/resource/Richard_Wagner,32
1,http://data.open.ac.uk/led/performance/http://...,G��tterd��mmerung,http://dbpedia.org/resource/Richard_Wagner,32
2,http://data.open.ac.uk/led/performance/http://...,Symphony No. 9,http://dbpedia.org/resource/Ludwig_van_Beethoven,32
3,http://data.open.ac.uk/led/performance/http://...,Aus mein Lieben,http://data.open.ac.uk/led/person/Smetana/1434...,32
4,http://data.open.ac.uk/led/performance/http://...,Charterhouse Suite,http://dbpedia.org/resource/Ralph_Vaughan_Will...,32
...,...,...,...,...
867,http://data.open.ac.uk/led/performance/b88ba9f...,Hungarian March,http://dbpedia.org/resource/Richard_Wagner,34
868,http://data.open.ac.uk/led/performance/87d23f9...,Introduction to Valse,http://dbpedia.org/resource/Hector_Berlioz,34
869,http://data.open.ac.uk/led/performance/87d23f9...,Introduction to Valse,http://dbpedia.org/resource/Richard_Wagner,34
870,http://data.open.ac.uk/led/performance/http://...,"The Planets, Op. 32 - Jupiter",http://dbpedia.org/resource/Gustav_Holst,33


## CQ14: What is the cause of a change, e.g. increasing number of concerts over time?

In [ ]:
#what was the cause of a change?
#number of performances per medium pre - war
query = """ PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
prefix foaf:<http://xmlns.com/foaf/0.1/>

select   ?medium (count(?lexp)as ?exp) 
WHERE {
  ?lexp event:time [ rdfs:label ?when ; edtf:century edtfdata:Century.19 ; edtf:yearOfCentury/rdf:value ?yoc ].
   ?lexp led:has_medium ?medium.
  FILTER ( ?yoc >= 32 && ?yoc <= 38 ) .   
} group by ?medium 
"""
# set the query
sparql_wd.setQuery(query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

In [ ]:
import pandas as pds
df_medium_pre = pds.DataFrame(results['results']['bindings'])
df_medium_pre.applymap(lambda x: x['value'])

,medium,exp
0,http://led.kmi.open.ac.uk/term/Medium.Broadcast,173
1,http://led.kmi.open.ac.uk/term/Medium.Live,309
2,http://led.kmi.open.ac.uk/term/Medium.Playback,39


In [ ]:
#number of performances per medium during war
query = """ PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
prefix foaf:<http://xmlns.com/foaf/0.1/>

select   ?medium (count(?lexp)as ?exp) 
WHERE {
  ?lexp event:time [ rdfs:label ?when ; edtf:century edtfdata:Century.19 ; edtf:yearOfCentury/rdf:value ?yoc ].
   ?lexp led:has_medium ?medium.
  FILTER ( ?yoc >= 39 && ?yoc <= 45 ) .   
} group by ?medium 
"""
# set the query
sparql_wd.setQuery(query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()
import pandas as pds
df_medium_war= pds.DataFrame(results['results']['bindings'])
df_medium_war.applymap(lambda x: x['value'])

,medium,exp
0,http://led.kmi.open.ac.uk/term/Medium.Live,121
1,http://led.kmi.open.ac.uk/term/Medium.Broadcast,13
2,http://led.kmi.open.ac.uk/term/Medium.Playback,15


In [ ]:
#number of performances per medium during war
query = """ PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
prefix foaf:<http://xmlns.com/foaf/0.1/>

select   ?medium (count(?lexp)as ?exp) 
WHERE {
  ?lexp event:time [ rdfs:label ?when ; edtf:century edtfdata:Century.19 ; edtf:yearOfCentury/rdf:value ?yoc ].
   ?lexp led:has_medium ?medium.
  FILTER ( ?yoc >= 46 && ?yoc <= 52 ) .   
} group by ?medium 
"""
# set the query
sparql_wd.setQuery(query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()
import pandas as pds
df_medium_after= pds.DataFrame(results['results']['bindings'])
df_medium_after.applymap(lambda x: x['value'])

,medium,exp
0,http://led.kmi.open.ac.uk/term/Medium.Live,134
1,http://led.kmi.open.ac.uk/term/Medium.Broadcast,14
2,http://led.kmi.open.ac.uk/term/Medium.Playback,8


## CQ 10 **QUERIES on brass bands**

We tried retrieving data on brass bands but we have found a few stuff . We have tried applying the research questions to this topic.



CQ10: How does the world of brass bands connect with that of elite music? <br>

In [ ]:
# retrieving the listening experiences of a brass band performance
query = """
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
  
  
SELECT DISTINCT ?lexp ?label ?date ?genre_label ?performer_label ?place_label
WHERE { 
  ?lexp event:time ?when.
  ?when rdfs:label ?date.
  ?lexp  dc:subject ?perf.
  ?lexp event:place ?place.
  ?place rdfs:label ?place_label. 
  ?perf rdfs:label ?label;
    	mo:genre ?genre;
    	mo:performer ?performer.
  ?genre rdfs:label ?genre_label.
  ?performer rdfs:label ?performer_label.
  
  FILTER(REGEX(?label, "brass band" , "i"))
  FILTER langMatches( lang(?place_label), "en" ) 
  FILTER langMatches( lang(?performer_label), "en" ) 
  FILTER langMatches( lang(?genre_label), "en" ) 
	 }
"""
# set the query
sparql_wd.setQuery(query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results_brass = sparql_wd.query().convert()


We don't seem to have brass band performances in the period of the 2nd world war (pre-during-after): ⬇

In [ ]:
# manipulate the result
for result in results_brass["results"]["bindings"]:
    print(result["label"]["value"]," DATE: ", result["date"]["value"]," PERFORMER: ",result["performer_label"]["value"]," GENRE: ",result["genre_label"]["value"]," PLACE: ",result["place_label"]["value"], "\n")

performance of 'New Orleans Brass Band music'  DATE:  1895  PERFORMER:  Buddy Bolden  GENRE:  Jazz  PLACE:  New Orleans 

performance of 'New Orleans Brass Band music'  DATE:  early 20th Century  PERFORMER:  Buddy Petit  GENRE:  Fanfare  PLACE:  New Orleans 

performance of 'New Orleans Brass Band music'  DATE:  early 20th Century  PERFORMER:  Buddy Petit  GENRE:  Flourish  PLACE:  New Orleans 

performance of 'New Orleans Brass Band music'  DATE:  early 20th Century  PERFORMER:  Frankie Duson  GENRE:  Fanfare  PLACE:  New Orleans 

performance of 'New Orleans Brass Band music'  DATE:  early 20th Century  PERFORMER:  Frankie Duson  GENRE:  Flourish  PLACE:  New Orleans 

performance of 'New Orleans Brass Band music'  DATE:  early 20th Century  PERFORMER:  Kid Rena  GENRE:  Fanfare  PLACE:  New Orleans 

performance of 'New Orleans Brass Band music'  DATE:  early 20th Century  PERFORMER:  Kid Rena  GENRE:  Flourish  PLACE:  New Orleans 

performance of 'New Orleans Brass Band music'  DA

In [ ]:
#retrieving the environment of performances
query= """
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
  
SELECT DISTINCT ?lexp ?label ?env
WHERE { 
  ?lexp event:time ?when.
  ?when rdfs:label ?date.
  ?lexp  dc:subject ?perf.
  ?perf rdfs:label ?label.
  ?lexp led:has_environment ?env.
  
  FILTER(REGEX(?label, "brass" , "i"))
  
	 }
"""
# set the query
sparql_wd.setQuery(query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results= sparql_wd.query().convert()


In [ ]:
# manipulate the result
for result in results["results"]["bindings"]:
    print(result["lexp"]["value"]," environment: ",result["env"]["value"], "\n")

http://data.open.ac.uk/led/lexp/1433408473434  environment:  http://led.kmi.open.ac.uk/term/ListeningEnvironment.Outdoor 

http://data.open.ac.uk/led/lexp/1382351150  environment:  http://led.kmi.open.ac.uk/term/ListeningEnvironment.Accompanied 

http://data.open.ac.uk/led/lexp/1382351150  environment:  http://led.kmi.open.ac.uk/term/ListeningEnvironment.Outdoor 

http://data.open.ac.uk/led/lexp/1382351150  environment:  http://led.kmi.open.ac.uk/term/ListeningEnvironment.Public 

http://data.open.ac.uk/led/lexp/1431469593713  environment:  http://led.kmi.open.ac.uk/term/ListeningEnvironment.Accompanied 

http://data.open.ac.uk/led/lexp/1431469593713  environment:  http://led.kmi.open.ac.uk/term/ListeningEnvironment.Outdoor 

http://data.open.ac.uk/led/lexp/1431469593713  environment:  http://led.kmi.open.ac.uk/term/ListeningEnvironment.Public 

http://data.open.ac.uk/led/lexp/1436017047829  environment:  http://led.kmi.open.ac.uk/term/ListeningEnvironment.Accompanied 

http://data.ope

CQ11: What did people ‘make of’, how did they react to, e.g. music?

In [ ]:
#retrieve texts for doing sentiment analysis
query = """
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
  
SELECT DISTINCT  ?label ?text ?date ?description
WHERE { 
  ?lexp event:time ?when.
  ?when rdfs:label ?date.
  ?lexp  dc:subject ?perf.
  ?perf rdfs:label ?label.
  ?lexp led:has_evidence_text ?text.
  FILTER(REGEX(?label, "brass band" , "i"))
}
"""
# set the query
sparql_wd.setQuery(query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results_text = sparql_wd.query().convert()


In [ ]:
# manipulate the result
for result in results_text["results"]["bindings"]:
    print(result["label"]["value"], "\n",result["text"]["value"], "\n")

performance of 'New Orleans Brass Band music' 
 Memories go back to the days of the loud jazz bands of the easy 'nineties [1890's], when we were just starting out with patch-up cornets and trombones, leaky valves on clarinets which were usually minus a key or two, and homemade bass fiddles, and warped guitars. We must have sounded just as lousy as the professional element, but by constant plugging night and day for days, months, and even years, some went on to fame and fortune (also ruin).  

performance of 'brass band' 
 The church-feast of its saint is of course for Anacapri, as for any self-respecting Italian town, the great day of the year���. As much of it as possible was squeezed all the morning, for St. Antony, into the piazzetta before the church, and as much more into that edifice as the robust odour mainly prevailing there allowed room for���. The shade outside was hot, and the sun was hot; but we waited as densely for him to come out, or rather to come ���on,��� as the pit a

In [ ]:
# SENTIMENT ANALYSIS:

In [ ]:
for result in results_text["results"]["bindings"]:
    q =array([result["text"]["value"]])
    q=vect.transform(q)
    print(result["label"]["value"], "\n",model.predict(q), " ->",[result["text"]["value"]])

performance of 'New Orleans Brass Band music' 
 ['neg']  -> ["Memories go back to the days of the loud jazz bands of the easy 'nineties [1890's], when we were just starting out with patch-up cornets and trombones, leaky valves on clarinets which were usually minus a key or two, and homemade bass fiddles, and warped guitars. We must have sounded just as lousy as the professional element, but by constant plugging night and day for days, months, and even years, some went on to fame and fortune (also ruin). "]
performance of 'brass band' 
 ['pos']  -> ['The church-feast of its saint is of course for Anacapri, as for any self-respecting Italian town, the great day of the year���. As much of it as possible was squeezed all the morning, for St. Antony, into the piazzetta before the church, and as much more into that edifice as the robust odour mainly prevailing there allowed room for���. The shade outside was hot, and the sun was hot; but we waited as densely for him to come out, or rather to

CQ12: Who was the author of a piece of music?


In [ ]:
# retrieve the composers of performances
query = """
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
  
SELECT DISTINCT  ?perf ?label ?author 
WHERE { 
  ?perf mo:composer ?author.
  ?perf rdfs:label ?label.
  FILTER(REGEX(?label, "brass band" , "i"))}
"""
# set the query
sparql_wd.setQuery(query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results_author = sparql_wd.query().convert()

In [ ]:
# manipulate the result
for result in results_author["results"]["bindings"]:
    print(result["label"]["value"],result["author"]["value"], "\n")
#not enough data for brass bads it seems

Walk��re for brass band http://dbpedia.org/resource/Richard_Wagner 



# extra queries

## sentiment analysis during war
 - all positive

In [ ]:
#retrieve texts for doing sentiment analysis
query_war = """
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
  
SELECT DISTINCT  ?songlabel ?text ?date ?description ?when ?yoc
WHERE { 
  ?lexp event:time [ rdfs:label ?when ; edtf:century edtfdata:Century.19 ; edtf:yearOfCentury/rdf:value ?yoc ].
  
  ?lexp  dc:subject ?perf.
  ?perf mo:performance_of ?song.
  ?song rdfs:label ?songlabel.
  ?lexp led:has_evidence_text ?text.
  FILTER ( ?yoc >= 39 && ?yoc <= 45 ) 
}
"""
# set the query
sparql_wd.setQuery(query_war)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results_war= sparql_wd.query().convert()

In [ ]:
# easier way to obtain a dataframe
import pandas as pds
df_text = pds.DataFrame(results_war['results']['bindings'])
df_text.applymap(lambda x: x['value'])

,songlabel,text,when,yoc
0,"Cello sonata n�� 1 in E minor, op. 38","One work, however, in the cellist���s repertor...",1945,45
1,Menuhin performs a concert for injured service...,For the troops stationed there life was not tr...,1945,45
2,Lincoln Portrait,Dear A\r\n\tJust a few words a propos of the L...,1943,43
3,La Marseillaise,"The reopening of the Op��ra, the first chance ...",1945,45
4,Fa��ade,My other extra mural activity at this time was...,1942,42
...,...,...,...,...
62,"Weinen, Klagen variations",This was the 1st recital in the rather restric...,"7 October, 1939",39
63,Piano Concerto,This was a concert promoted by the S.C.R. and ...,"13 April, 1940",40
64,Symphony No. 5,This was a concert promoted by the S.C.R. and ...,"13 April, 1940",40
65,Symphony No.16,This was a concert promoted by the S.C.R. and ...,"13 April, 1940",40


In [ ]:
df_sent_2 = pd.DataFrame(columns=['songlabel','year','sentiment'])
idx=0
for result in results_war["results"]["bindings"]:   
    v =array([result["text"]["value"]])
    v =vect.transform(v)
    sent=str(model.predict(v)).replace("[","").replace("]","")
    df_sent_2.loc[idx] = [result["songlabel"]["value"],result["yoc"]["value"],sent]
    idx+=1
df_sent_2

,songlabel,year,sentiment
0,"Cello sonata n�� 1 in E minor, op. 38",45,'pos'
1,Menuhin performs a concert for injured service...,45,'pos'
2,Lincoln Portrait,43,'pos'
3,La Marseillaise,45,'pos'
4,Fa��ade,42,'pos'
...,...,...,...
62,"Weinen, Klagen variations",39,'pos'
63,Piano Concerto,40,'pos'
64,Symphony No. 5,40,'pos'
65,Symphony No.16,40,'pos'


In [ ]:
list(df_sent_2["sentiment"]) #all positive

## Retrieving places
We have to decide if we want focus on the area of London/ England or represent a wider area. Cause we have also list. exp. outide England.

In [ ]:
#retrieving the 10 most frequent leistening experiences' places on the database. we could apply the time filter if we want to use this
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
prefix foaf:<http://xmlns.com/foaf/0.1/>

select  ?place (count(?lexp)as ?exp)
WHERE {
   ?lexp event:time ?when;
    event:place ?place.
   ?lexp  dc:subject ?perf.
   ?perf rdfs:label ?perf_label; 
    mo:genre ?genre.

  ?when rdfs:label ?date.
   
} group by ?place order by desc(?exp) limit 10

## Focusing on live performances during the afterwar period

In [ ]:
# retrieving live performances post 2w war  #delete?
query = """
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX edtf: <http://data.open.ac.uk/edtf/ontology/>
PREFIX edtfdata: <http://data.open.ac.uk/time/edtf/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX led: <http://led.kmi.open.ac.uk/term/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
prefix mo:<http://purl.org/ontology/mo/>
prefix db:<http://dbpedia.org/ontology/>
prefix uk:<http://data.open.ac.uk/ukresearchproject/ontology/>
prefix ox:<http://vocab.ox.ac.uk/projectfunding#>
  
SELECT DISTINCT  ?perf ?label ?genre_label ?performer_label
WHERE { 
  ?lexp led:has_medium led:Medium.Live.
  ?lexp event:time [ rdfs:label ?when ; edtf:century edtfdata:Century.19 ; edtf:yearOfCentury/rdf:value ?yoc ].
  ?lexp  dc:subject ?perf.
  ?perf rdfs:label ?label;
    mo:genre ?genre;
    	mo:performer ?performer.
  ?genre rdfs:label ?genre_label.
  ?performer rdfs:label ?performer_label.
   FILTER ( ?yoc >=46 && ?yoc <= 52 ) 
  }
"""
# set the query
sparql_wd.setQuery(query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results= sparql_wd.query().convert()
# manipulate the result
for result in results["results"]["bindings"]:
    print(result["label"]["value"]," GENRE: ", result["genre_label"]["value"], " PERFORMER: ", result["performer_label"]["value"])

# since the research is focused on non 'elite' music,
# we could search for performances excluding the ones of Classical Music/Opera/Simphony/Chamber music
#  (I think these genres could be defined 'elite' music since they were listened by the most rich/educated people -> 'elite' )
# we could focus on Blues music/rock/country for example

performance of 'unspecified orchestral music'  GENRE:  Classical music  PERFORMER:  Richard Strauss
performance of 'unspecified orchestral music'  GENRE:  Classical music  PERFORMER:  Richard Strauss
performance of 'unspecified orchestral music'  GENRE:  Classical music  PERFORMER:  Richard Strauss
performance of 'unspecified orchestral music'  GENRE:  Classical music  PERFORMER:  Richard Strauss
performance of 'unspecified orchestral music'  GENRE:  Classical music  PERFORMER:  Richard Strauss (conductor)
performance of 'unspecified orchestral music'  GENRE:  Classical music  PERFORMER:  Richard Strauss (conductor)
performance of 'unspecified orchestral music'  GENRE:  Classical music  PERFORMER:  Royal Philharmonic Orchestra
performance of 'unspecified orchestral music'  GENRE:  Classical music  PERFORMER:  Royal Philharmonic Orchestra
performance of 'unspecified orchestral music'  GENRE:  Classical music  PERFORMER:  Royal Philharmonic Orchestra
performance of 'unspecified orchestra